In [1]:
import pandas as pd
import os, glob, re

# Naming conventions

In [2]:
all_jpgs= glob.glob('/cs/student/projects4/ml/2023/asrivast/datasets/dress3_16/input/*.JPG')
print(f"total images: {len(all_jpgs)}")

total images: 139


In [17]:
import PIL.ExifTags as ExifTags


In [52]:
def get_exif(image_path):
    image = Image.open(image_path)
    exif_data = image._getexif()
    if not exif_data:
        return {}

    exif = {}
    for tag, value in exif_data.items():
        decoded = ExifTags.TAGS.get(tag, tag)
        exif[decoded] = value
    return exif

def get_exif_gps(exif_info: dict):
    gps_info = {}

    for key in exif_info['GPSInfo'].keys():
        decode = ExifTags.GPSTAGS.get(key,key)
        gps_info[decode] = exif_info['GPSInfo'][key]#int.from_bytes(exif_info['GPSInfo'][key], byteorder='big')
    return gps_info


exif_info = get_exif(all_jpgs[125])
for tag, value in exif_info.items():

    if tag == 'GPSInfo':
        print("GPSInfo")
        gps_info = get_exif_gps(exif_info)
        for tag, value in gps_info.items():
            print(f"{' ' * 5}{tag}: {value}")

    else:
        print(f"{tag}: {value}")
    

ResolutionUnit: 2
ExifOffset: 348
Make: Canon
Model: Canon EOS 1100D
YResolution: 72.0
Orientation: 8
DateTime: 2024:07:04 12:44:48
YCbCrPositioning: 2
Copyright: 
XResolution: 72.0
Artist: 
ExifVersion: b'0230'
ComponentsConfiguration: b'\x01\x02\x03\x00'
ShutterSpeedValue: 0.0
DateTimeOriginal: 2024:07:04 12:44:48
DateTimeDigitized: 2024:07:04 12:44:48
ApertureValue: 7.375
ExposureBiasValue: 0.0
MeteringMode: 2
UserComment: b'\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00

In [54]:

# make a dataframe with 3 columns
camera_names_df= pd.DataFrame(columns=['tower', 'aim', 'name', 'focal_length', 'Width', 'Height', 'Camera_model'])


for i, jpg in enumerate(all_jpgs):
    camera_name= re.split('[-_]', os.path.basename(jpg))[2]
    exif_info = get_exif(jpg)
    
    # Create a dictionary with the new data
    camera_names_df.loc[len(camera_names_df)] = {
        'tower': camera_name[0],
        'aim': camera_name[1],
        'name': camera_name[2],
        'focal_length': exif_info['FocalLength'],
        'Width': exif_info['ExifImageWidth'],
        'Height': exif_info['ExifImageHeight'],
        'Camera_model': exif_info['LensModel']
    }

    # if camera_name == 'KF1':
    #     print(i, exif_info['LensModel'])
    
# change i to I
camera_names_df['tower']= camera_names_df['tower'].str.upper()


In [55]:
camera_names_df.to_csv('camera_names.csv', index=False)

In [56]:
# group by  width and height then focal length

group_df= pd.pivot_table(camera_names_df, index=['Width', 'Height', 'focal_length'], aggfunc='count')
display(group_df)

Camera_model  aim  name  tower
Width Height focal_length                                
4272  2848   27.0                     1    1     1      1
5184  3456   50.0                     2    2     2      2
6240  4160   35.0                     4    4     4      4
             50.0                    68   68    68     68
             78.0                     1    1     1      1
             85.0                     8    8     8      8
             91.0                     1    1     1      1
             94.0                     1    1     1      1
             98.0                     8    8     8      8
             122.0                    2    2     2      2
             126.0                   11   11    11     11
             131.0                   22   22    22     22
             135.0                    4    4     4      4
             140.0                    2    2     2      2
             200.0                    4    4     4      4

In [34]:
camera_names_df[((camera_names_df["tower"]=="A") & (camera_names_df["aim"]=="B"))]

,tower,aim,name,focal_length
0,A,B,1,50.0
1,A,B,2,50.0
2,A,B,3,50.0
3,A,B,4,50.0
4,A,B,5,50.0
5,A,B,6,35.0
6,A,B,7,50.0
7,A,B,M,50.0
